##### 著作権 2018 TensorFlow 著者。

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# トレーニングのチェックポイント

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/checkpoint"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org で見る</a>   </td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab で実行する</a>   </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">GitHub でソースを表示</a>   </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/guide/checkpoint.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">ノートブックをダウンロード</a>   </td>
</table>

「TensorFlow モデルを保存する」というフレーズは通常、次の 2 つのいずれかを意味します。

1. チェックポイント、または
2. 保存されたモデル。

チェックポイントは、モデルで使用されるすべてのパラメーター ( `tf.Variable`オブジェクト) の正確な値をキャプチャします。チェックポイントには、モデルによって定義された計算の記述が含まれていないため、通常は、保存されたパラメーター値を使用するソース コードが利用可能な場合にのみ役立ちます。

一方、SavedModel 形式には、パラメーター値 (チェックポイント) に加えて、モデルによって定義された計算のシリアル化された記述が含まれます。この形式のモデルは、モデルを作成したソース コードから独立しています。したがって、これらは TensorFlow Serving、TensorFlow Lite、TensorFlow.js、または他のプログラミング言語 (C、C++、Java、Go、Rust、C# などの TensorFlow API) のプログラムを介したデプロイメントに適しています。

このガイドでは、チェックポイントの書き込みと読み取りのための API について説明します。

In [0]:
import tensorflow as tf

In [0]:
class Net(tf.keras.Model):
  """A simple linear model."""

  def __init__(self):
    super(Net, self).__init__()
    self.l1 = tf.keras.layers.Dense(5)

  def call(self, x):
    return self.l1(x)

In [0]:
net = Net()

## `tf.keras`トレーニング API からの保存

[保存と復元については`tf.keras`ガイド](./keras/overview.ipynb#save_and_restore)を参照してください。

`tf.keras.Model.save_weights` TensorFlow チェックポイントを保存します。 

In [0]:
net.save_weights('easy_checkpoint')

## チェックポイントの書き込み


TensorFlow モデルの永続的な状態は`tf.Variable`オブジェクトに保存されます。これらは直接構築できますが、多くの場合、 `tf.keras.layers`や`tf.keras.Model`などの高レベル API を通じて作成されます。

変数を管理する最も簡単な方法は、変数を Python オブジェクトにアタッチし、それらのオブジェクトを参照することです。

`tf.train.Checkpoint` 、 `tf.keras.layers.Layer` 、および`tf.keras.Model`のサブクラスは、その属性に割り当てられた変数を自動的に追跡します。次の例では、単純な線形モデルを構築し、モデルのすべての変数の値を含むチェックポイントを書き込みます。

`Model.save_weights`を使用すると、モデルチェックポイントを簡単に保存できます。

### 手動チェックポイント設定

#### 設定

`tf.train.Checkpoint`のすべての機能をデモンストレーションするために、おもちゃのデータセットと最適化ステップを定義します。

In [0]:
def toy_dataset():
  inputs = tf.range(10.)[:, None]
  labels = inputs * 5. + tf.range(5.)[None, :]
  return tf.data.Dataset.from_tensor_slices(
    dict(x=inputs, y=labels)).repeat().batch(2)

In [0]:
def train_step(net, example, optimizer):
  """Trains `net` on `example` using `optimizer`."""
  with tf.GradientTape() as tape:
    output = net(example['x'])
    loss = tf.reduce_mean(tf.abs(output - example['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))
  return loss

#### チェックポイント オブジェクトを作成する

チェックポイントを手動で作成するには、 `tf.train.Checkpoint`オブジェクトが必要です。チェックポイントを作成するオブジェクトがオブジェクトの属性として設定される場所。

`tf.train.CheckpointManager`は、複数のチェックポイントの管理にも役立ちます。

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

#### モデルをトレーニングしてチェックポイントを作成する

次のトレーニング ループは、モデルとオプティマイザーのインスタンスを作成し、それらを`tf.train.Checkpoint`オブジェクトに収集します。データの各バッチに対してループ内のトレーニング ステップを呼び出し、定期的にチェックポイントをディスクに書き込みます。

In [0]:
def train_and_checkpoint(net, manager):
  ckpt.restore(manager.latest_checkpoint)
  if manager.latest_checkpoint:
    print("Restored from {}".format(manager.latest_checkpoint))
  else:
    print("Initializing from scratch.")

  for _ in range(50):
    example = next(iterator)
    loss = train_step(net, example, opt)
    ckpt.step.assign_add(1)
    if int(ckpt.step) % 10 == 0:
      save_path = manager.save()
      print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))
      print("loss {:1.2f}".format(loss.numpy()))

In [0]:
train_and_checkpoint(net, manager)

#### 回復してトレーニングを継続する

最初の後に、新しいモデルとマネージャーを渡すことができますが、中断したところからトレーニングを開始します。

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
dataset = toy_dataset()
iterator = iter(dataset)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=net, iterator=iterator)
manager = tf.train.CheckpointManager(ckpt, './tf_ckpts', max_to_keep=3)

train_and_checkpoint(net, manager)

`tf.train.CheckpointManager`オブジェクトは古いチェックポイントを削除します。上記では、最新の 3 つのチェックポイントのみを保持するように構成されています。

In [0]:
print(manager.checkpoints)  # List the three remaining checkpoints

これらのパス (例: `'./tf_ckpts/ckpt-10'`は、ディスク上のファイルではありません。代わりに、それらは`index`ファイルと、変数値を含む 1 つ以上のデータ ファイルのプレフィックスになります。これらのプレフィックスは、 `CheckpointManager`がその状態を保存する単一の`checkpoint`ファイル ( `'./tf_ckpts/checkpoint'` ) にグループ化されます。

In [0]:
!ls ./tf_ckpts

<a id="loading_mechanics"></a>

## ローディング機構

TensorFlow は、ロードされるオブジェクトから開始して、名前付きエッジを持つ有向グラフを走査することにより、変数をチェックポイントされた値と照合します。エッジ名は通常、オブジェクトの属性名から取得されます (例: `self.l1 = tf.keras.layers.Dense(5)`の`"l1"` )。 `tf.train.Checkpoint` `tf.train.Checkpoint(step=...)`の`"step"`のように、キーワード引数名​​を使用します。

上記の例の依存関係グラフは次のようになります。

![サンプルトレーニングループの依存関係グラフの視覚化](https://tensorflow.org/images/guide/whole_checkpoint.svg)

オプティマイザーは赤、通常変数は青、オプティマイザー スロット変数はオレンジ色です。他のノード (たとえば`tf.train.Checkpoint`を表す) は黒です。

スロット変数はオプティマイザの状態の一部ですが、特定の変数用に作成されます。たとえば、上の`'m'`エッジは運動量に対応し、Adam オプティマイザーは変数ごとに追跡します。スロット変数は、変数とオプティマイザの両方が保存される場合にのみチェックポイントに保存されるため、破線のエッジになります。

`tf.train.Checkpoint`オブジェクトで`restore()`を呼び出すと、要求された復元がキューに入れられ、 `Checkpoint`オブジェクトから一致するパスが見つかるとすぐに変数値が復元されます。たとえば、ネットワークとレイヤーを介してモデルへの 1 つのパスを再構築することで、上で定義したモデルからバイアスだけを読み込むことができます。

In [0]:
to_restore = tf.Variable(tf.zeros([5]))
print(to_restore.numpy())  # All zeros
fake_layer = tf.train.Checkpoint(bias=to_restore)
fake_net = tf.train.Checkpoint(l1=fake_layer)
new_root = tf.train.Checkpoint(net=fake_net)
status = new_root.restore(tf.train.latest_checkpoint('./tf_ckpts/'))
print(to_restore.numpy())  # We get the restored value now

これらの新しいオブジェクトの依存関係グラフは、上で書いた大きなチェックポイントのより小さなサブグラフです。これには、 `tf.train.Checkpoint`チェックポイントに番号を付けるために使用するバイアスと保存カウンターのみが含まれています。

![バイアス変数のサブグラフの視覚化](https://tensorflow.org/images/guide/partial_checkpoint.svg)

`restore()`は、オプションのアサーションを含むステータス オブジェクトを返します。新しい`Checkpoint`で作成したすべてのオブジェクトが復元されたため、 `status.assert_existing_objects_matched()`が成功します。

In [0]:
status.assert_existing_objects_matched()

チェックポイントには、レイヤーのカーネルやオプティマイザーの変数など、一致していないオブジェクトが多数あります。 `status.assert_consumed()`チェックポイントとプログラムが正確に一致する場合にのみ合格し、ここで例外をスローします。

### 修復の遅れ

TensorFlow の`Layer`オブジェクトは、入力シェイプが利用可能な場合、変数の作成が最初の呼び出しまで遅れることがあります。たとえば、 `Dense`レイヤーのカーネルの形状はレイヤーの入力形状と出力形状の両方に依存するため、コンストラクターの引数として必要な出力形状は、変数を独自に作成するのに十分な情報ではありません。 `Layer`を呼び出すと変数の値も読み取られるため、変数の作成と最初の使用の間に復元が行われる必要があります。

このイディオムをサポートするために、 `tf.train.Checkpoint`は、一致する変数をまだ持たないリストアをキューに入れます。

In [0]:
delayed_restore = tf.Variable(tf.zeros([1, 5]))
print(delayed_restore.numpy())  # Not restored; still zeros
fake_layer.kernel = delayed_restore
print(delayed_restore.numpy())  # Restored

### チェックポイントを手動で検査する

`tf.train.list_variables`チェックポイント キーとチェックポイント内の変数の形状をリストします。チェックポイント キーは、上に表示されたグラフ内のパスです。

In [0]:
tf.train.list_variables(tf.train.latest_checkpoint('./tf_ckpts/'))

### リストと辞書の追跡

`self.l1 = tf.keras.layers.Dense(5)`のような直接の属性割り当てと同様に、リストと辞書を属性に割り当てると、その内容が追跡されます。

In [0]:
save = tf.train.Checkpoint()
save.listed = [tf.Variable(1.)]
save.listed.append(tf.Variable(2.))
save.mapped = {'one': save.listed[0]}
save.mapped['two'] = save.listed[1]
save_path = save.save('./tf_list_example')

restore = tf.train.Checkpoint()
v2 = tf.Variable(0.)
assert 0. == v2.numpy()  # Not restored yet
restore.mapped = {'two': v2}
restore.restore(save_path)
assert 2. == v2.numpy()

リストと辞書のラッパー オブジェクトに気づくかもしれません。これらのラッパーは、基礎となるデータ構造のチェックポイント可能なバージョンです。属性ベースの読み込みと同様に、これらのラッパーは変数がコンテナに追加されるとすぐに変数の値を復元します。

In [0]:
restore.listed = []
print(restore.listed)  # ListWrapper([])
v1 = tf.Variable(0.)
restore.listed.append(v1)  # Restores v1, from restore() in the previous cell
assert 1. == v1.numpy()

同じ追跡が`tf.keras.Model`のサブクラスに自動的に適用され、たとえばレイヤーのリストを追跡するために使用できます。

## Estimator を使用したオブジェクトベースのチェックポイントの保存

[「推定ガイド」を](https://www.tensorflow.org/guide/estimator)参照してください。

デフォルトでは、エスティメーターは、前のセクションで説明したオブジェクト グラフではなく、変数名を使用してチェックポイントを保存します。 `tf.train.Checkpoint`名前ベースのチェックポイントを受け入れますが、モデルの一部を Estimator の`model_fn`の外に移動すると変数名が変更される可能性があります。オブジェクトベースのチェックポイントを保存すると、Estimator 内でモデルをトレーニングし、それを Estimator の外で使用することが容易になります。

In [0]:
import tensorflow.compat.v1 as tf_compat

In [0]:
def model_fn(features, labels, mode):
  net = Net()
  opt = tf.keras.optimizers.Adam(0.1)
  ckpt = tf.train.Checkpoint(step=tf_compat.train.get_global_step(),
                             optimizer=opt, net=net)
  with tf.GradientTape() as tape:
    output = net(features['x'])
    loss = tf.reduce_mean(tf.abs(output - features['y']))
  variables = net.trainable_variables
  gradients = tape.gradient(loss, variables)
  return tf.estimator.EstimatorSpec(
    mode,
    loss=loss,
    train_op=tf.group(opt.apply_gradients(zip(gradients, variables)),
                      ckpt.step.assign_add(1)),
    # Tell the Estimator to save "ckpt" in an object-based format.
    scaffold=tf_compat.train.Scaffold(saver=ckpt))

tf.keras.backend.clear_session()
est = tf.estimator.Estimator(model_fn, './tf_estimator_example/')
est.train(toy_dataset, steps=10)

`tf.train.Checkpoint` `model_dir`から Estimator のチェックポイントをロードできます。

In [0]:
opt = tf.keras.optimizers.Adam(0.1)
net = Net()
ckpt = tf.train.Checkpoint(
  step=tf.Variable(1, dtype=tf.int64), optimizer=opt, net=net)
ckpt.restore(tf.train.latest_checkpoint('./tf_estimator_example/'))
ckpt.step.numpy()  # From est.train(..., steps=10)

## まとめ

TensorFlow オブジェクトは、使用する変数の値を保存および復元するための簡単な自動メカニズムを提供します。
